In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.api.types import is_string_dtype
from typing import List, Set, Dict, Tuple, Optional, Union
from nltk.tokenize import word_tokenize 
import ftfy
import textacy
from textacy import preprocess
import torchtext
from torchtext.data import Field
from utils import io
from torchtext.data import Field, Dataset, Example
import csv


In [6]:
              
def prepare_csv(csv_file:str, max_n_labels, path='MBTI', sep='\t', text_col:int=1, label_cols:int=[0],header=None):
    df = pd.read_csv(Path(path)/csv_file, sep=sep, header=header)
    df[1] = df[text_col].str.replace("\n", " ")
    df[1]=df[1].apply(ftfy.fix_text)
    df[1]=df[1].apply(lambda text: textacy.preprocess_text(text,lowercase=False, no_currency_symbols=True,no_urls=True,no_emails=True,no_phone_numbers=True,no_accents=True,no_contractions=False))
    df[1] = df[1].apply(lambda text: text.replace('\"', ''))
    df[1] = df[1].apply(lambda text: text.replace('"', ''))
    df[1] = df[1].apply(lambda text: ' '.join(text.split()).strip())
    df[1]=df[1].apply(ftfy.fix_text)
    df[0] = df.apply(lambda row: ''.join([str(lbl) for lbl in row[label_cols]]), axis=1)
    df = df[[0, 1]]
    df.to_csv(Path(path)/'tmp.tsv', sep='\t', header=False, index=False, index_label=None)
    df[0]=df[0].astype('str')
    df[0]=df[0].apply(lambda x: x if len(x) == max_n_labels else ''.join(['0' for i in range(4-len(x))])+str(x))


def to_tsv(outfpath, labels:list, texts:list):
    with open(outfpath, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        for label, text in zip(labels, texts):
            writer.writerow([label, text])

In [24]:
df = pd.read_csv('new.tsv', header=None, sep='\t')